# CPS Report

### Preliminary installations with ROS Noetic/Ubuntu 20.04 

https://docs.px4.io/main/en/dev_setup/dev_env_linux_ubuntu.html

In [ ]:
# download PX4
!git clone https://github.com/PX4/PX4-Autopilot.git --recursive

# install
!bash ./PX4-Autopilot/Tools/setup/ubuntu.sh -no-sim-tools --no-nuttx

# then restart computer

# additional dependencies
!sudo apt-get install protobuf-compiler libeigen3-dev libopencv-dev -y